# Compute graph kernels in Python

In [1]:
import networkx as nx
import numpy as np
import logging
import scipy as sp
from math import pi
import sys

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

import os 
dir_path = os.path.dirname(os.path.realpath('__file__'))

DEBUG:root:test


#### Import kernel functions from diffuPy

The kernel functions a imported from the package. Despite this the functions implementation are in this notebook (final _imp in the function name).

In [2]:
from diffupy.kernel import commute_time_kernel, p_step_kernel, inverse_cosine_kernel, diffusion_kernel, regularised_laplacian_kernel

### Import example graph

In [3]:
G = nx.read_gml(dir_path+'/04_unit_testing/_graph.gml', label='id')
G

### General functions

#### Kernel functions

In [4]:
def get_laplacian(G, normalized = False):
    if nx.is_directed(G):
        sys.exit('Graph must be undirected')
        
    if not normalized:
        L = nx.laplacian_matrix(G).toarray()
    else:
        L = nx.normalized_laplacian_matrix(G).toarray()
    
    return L

In [5]:
def set_diagonal_matrix(M, d):
    for j, row in enumerate(M):
        for i, x in enumerate(row):
            if i==j:
                M[j][i] = d[i]
            else:
                M[j][i] = x
    return M

#### Tests functions 

In [6]:
def csv_labeled_matrix_to_nparray(path):
    # Import matrix from csv file and remove headers
    m = np.genfromtxt(path, delimiter=',')
    return np.array([[x for x in a if ~np.isnan(x)] for a in m[1:]])

In [7]:
def run_kernel_test(kernel_func, G, T):
    M = kernel_func(G)
    logging.info(' %s  \n %s\n', 'Computed matrix', M)
    logging.info(' %s  \n %s\n', 'Test matrix', T)
    # Assert rounded similarity (floating comma)
    assert np.allclose(M, T)
    logging.info(' Test '+ kernel_func.__name__ +' passed')

## Commute time kernel

Computes the conmute-time kernel, which is the expected time of going back and forth between a couple of nodes. If the network is connected, then the commute time kernel will be totally dense, therefore reflecting global properties of the network. For further details, see [Yen, 2007]. This kernel can be computed using both the unnormalised and normalised graph Laplacian.

In [8]:
def commute_time_kernel_impl(G, normalized = False):
    # Apply pseudo-inverse (moore-penrose) of laplacian matrix
    return np.linalg.pinv(get_laplacian(G, normalized))

run_kernel_test(commute_time_kernel, G, csv_labeled_matrix_to_nparray(dir_path+'/04_unit_testing/commuteTimeKernel.csv'))

INFO:root: Computed matrix  
 [[ 0.13585449  0.01349958  0.00707249 ...  0.00140383 -0.00238913
  -0.00211733]
 [ 0.01349958  0.09948265  0.00573008 ...  0.0325679  -0.00379415
   0.00052156]
 [ 0.00707249  0.00573008  0.05223533 ... -0.00223114  0.01331363
  -0.0036509 ]
 ...
 [ 0.00140383  0.0325679  -0.00223114 ...  0.39279541 -0.00579927
  -0.00742603]
 [-0.00238913 -0.00379415  0.01331363 ... -0.00579927  0.37149228
  -0.00921487]
 [-0.00211733  0.00052156 -0.0036509  ... -0.00742603 -0.00921487
   0.35492096]]

INFO:root: Test matrix  
 [[ 0.13585449  0.01349958  0.00707249 ...  0.00140383 -0.00238913
  -0.00211733]
 [ 0.01349958  0.09948265  0.00573008 ...  0.0325679  -0.00379415
   0.00052156]
 [ 0.00707249  0.00573008  0.05223533 ... -0.00223114  0.01331363
  -0.0036509 ]
 ...
 [ 0.00140383  0.0325679  -0.00223114 ...  0.39279541 -0.00579927
  -0.00742603]
 [-0.00238913 -0.00379415  0.01331363 ... -0.00579927  0.37149228
  -0.00921487]
 [-0.00211733  0.00052156 -0.0036509  ...

## Diffusion kernel

Computes the classical diffusion kernel that involves matrix exponentiation. It has a "bandwidth" parameter σ^2 that controls the extent of the spreading. Quoting [Smola, 2003]: K(x1,x2) can be visualized as the quantity of some substance that would accumulate at vertex x2 after a given amount of time if we injected the substance at vertex x1 and let it diffuse through the graph along the edges. This kernel can be computed using both the unnormalised and normalised graph Laplacian.

In [9]:
def diffusion_kernel_imp(G, sigma2 = 1, normalized = True):    
    EL = -sigma2/2*get_laplacian(G, normalized)
    return sp.linalg.expm(EL)

run_kernel_test(diffusion_kernel, G, csv_labeled_matrix_to_nparray(dir_path+'/04_unit_testing/diffusionKernel.csv'))

INFO:root: Computed matrix  
 [[6.12865235e-01 3.29082231e-02 2.45247668e-02 ... 1.44983399e-03
  7.83151051e-04 8.51056826e-04]
 [3.29082231e-02 6.19159113e-01 2.22086964e-02 ... 5.11968484e-02
  6.85685184e-04 1.58484076e-03]
 [2.45247668e-02 2.22086964e-02 6.21984101e-01 ... 1.02408503e-03
  3.70359480e-02 6.01032388e-04]
 ...
 [1.44983399e-03 5.11968484e-02 1.02408503e-03 ... 6.17974062e-01
  9.51450956e-05 8.66444427e-05]
 [7.83151051e-04 6.85685184e-04 3.70359480e-02 ... 9.51450956e-05
  6.14837957e-01 3.00128639e-05]
 [8.51056826e-04 1.58484076e-03 6.01032388e-04 ... 8.66444427e-05
  3.00128639e-05 6.11799841e-01]]

INFO:root: Test matrix  
 [[6.12865235e-01 3.29082231e-02 2.45247668e-02 ... 1.44983399e-03
  7.83151051e-04 8.51056826e-04]
 [3.29082231e-02 6.19159113e-01 2.22086964e-02 ... 5.11968484e-02
  6.85685184e-04 1.58484076e-03]
 [2.45247668e-02 2.22086964e-02 6.21984101e-01 ... 1.02408503e-03
  3.70359480e-02 6.01032388e-04]
 ...
 [1.44983399e-03 5.11968484e-02 1.0240850

## Inverse cosine kernel

Computes the inverse cosine kernel, which is based on a cosine transform on the spectrum of the normalized Laplacian matrix. Quoting [Smola, 2003]: the inverse cosine kernel treats lower complexity functions almost equally, with a significant reduction in the upper end of the spectrum. This kernel is computed using the normalised graph Laplacian.

In [10]:
def inverse_cosine_kernel_imp(G):    
    # Decompose matrix (Singular Value Decomposition)
    U, S, _ = np.linalg.svd(get_laplacian(G, normalized = True)*(pi/4))

    return np.matmul(np.matmul(U, np.diag(np.cos(S))), np.transpose(U))
    
run_kernel_test(inverse_cosine_kernel, G, csv_labeled_matrix_to_nparray(dir_path+'/04_unit_testing/inverseCosineKernel.csv'))

INFO:root: Computed matrix  
 [[ 6.89153644e-01  5.06788574e-02  3.42575829e-02 ... -4.17444129e-03
  -2.22597740e-03 -2.38465091e-03]
 [ 5.06788574e-02  6.71392147e-01  2.20078787e-02 ...  8.97968412e-02
  -2.01160492e-03 -4.58819509e-03]
 [ 3.42575829e-02  2.20078787e-02  6.63386784e-01 ... -3.09491030e-03
   6.46799766e-02 -1.77647780e-03]
 ...
 [-4.17444129e-03  8.97968412e-02 -3.09491030e-03 ...  6.74893062e-01
  -3.15696738e-04 -3.25294979e-04]
 [-2.22597740e-03 -2.01160492e-03  6.46799766e-02 ... -3.15696738e-04
   6.83686983e-01 -6.82799209e-05]
 [-2.38465091e-03 -4.58819509e-03 -1.77647780e-03 ... -3.25294979e-04
  -6.82799209e-05  6.92214032e-01]]

INFO:root: Test matrix  
 [[ 6.89153644e-01  5.06788574e-02  3.42575829e-02 ... -4.17444129e-03
  -2.22597740e-03 -2.38465091e-03]
 [ 5.06788574e-02  6.71392147e-01  2.20078787e-02 ...  8.97968412e-02
  -2.01160492e-03 -4.58819509e-03]
 [ 3.42575829e-02  2.20078787e-02  6.63386784e-01 ... -3.09491030e-03
   6.46799766e-02 -1.776477

## p Step Kernel

Computes the p-step random walk kernel. This kernel is more focused on local properties of the nodes, because random walks are limited in terms of length. Therefore, if p is small, only a fraction of the values K(x1,x2) will be non-null if the network is sparse [Smola, 2003]. The parameter a is a regularising term that is summed to the spectrum of the normalised Laplacian matrix, and has to be 2 or greater. The p-step kernels can be cheaper to compute and have been successful in biological tasks, see the benchmark in [Valentini, 2014].


In [11]:
def p_step_kernel_imp(G, a = 2, p = 5):
    minusL = -get_laplacian(G, normalized = True)
    
    # Not optimal but kept for clarity
    # here we restrict to the normalised version, as the eigenvalues are
    # between 0 and 2 -> restriction a >= 2
    if a < 2:
        sys.exit('Eigenvalues must be between 0 and 2')
    if p < 0:
        sys.exit('p must be greater than 0')
                
    M = set_diagonal_matrix(minusL, [x + a for x in np.diag(minusL)])

    if p == 1: return M
    
    return np.linalg.matrix_power(M, p)

run_kernel_test(p_step_kernel, G, csv_labeled_matrix_to_nparray(dir_path+'/04_unit_testing/pStepKernel.csv'))

INFO:root: Computed matrix  
 [[2.08076205 1.15956073 1.12628329 ... 0.3537027  0.22148895 0.20527764]
 [1.15956073 3.11445732 1.30505581 ... 1.39693131 0.22230188 0.38745453]
 [1.12628329 1.30505581 3.6974033  ... 0.37356627 1.06904579 0.26027947]
 ...
 [0.3537027  1.39693131 0.37356627 ... 2.7254085  0.10096354 0.08424824]
 [0.22148895 0.22230188 1.06904579 ... 0.10096354 2.24207215 0.03899635]
 [0.20527764 0.38745453 0.26027947 ... 0.08424824 0.03899635 1.79935198]]

INFO:root: Test matrix  
 [[2.08076205 1.15956073 1.12628329 ... 0.3537027  0.22148895 0.20527764]
 [1.15956073 3.11445732 1.30505581 ... 1.39693131 0.22230188 0.38745453]
 [1.12628329 1.30505581 3.6974033  ... 0.37356627 1.06904579 0.26027947]
 ...
 [0.3537027  1.39693131 0.37356627 ... 2.7254085  0.10096354 0.08424824]
 [0.22148895 0.22230188 1.06904579 ... 0.10096354 2.24207215 0.03899635]
 [0.20527764 0.38745453 0.26027947 ... 0.08424824 0.03899635 1.79935198]]

INFO:root: Test p_step_kernel passed


## Regularised Laplacian Kernel

Computes the regularised Laplacian kernel, which is a standard in biological networks. The regularised Laplacian kernel arises in numerous situations, such as the finite difference formulation of the diffusion equation and in Gaussian process estimation. Sticking to the heat diffusion model, this function allows to control the constant terms summed to the diagonal through add_diag, i.e. the strength of the leaking in each node. If a node has diagonal term of 0, it is not allowed to disperse heat. The larger the diagonal term of a node, the stronger the first order heat dispersion in it, provided that it is positive. Every connected component in the graph should be able to disperse heat, i.e. have at least a node i with add_diag[i] > 0. If this is not the case, the result diverges. More details on the parameters can be found in [Smola, 2003]. This kernel can be computed using both the unnormalised and normalised graph Laplacian.

In [12]:
def regularised_laplacian_kernel_imp(G, sigma2 = 1, add_diag = 1, normalized = False):
    L = get_laplacian(G, normalized)
    RL = set_diagonal_matrix(sigma2*L, [x + add_diag for x in np.diag(L)])
        
    return np.linalg.inv(RL)

run_kernel_test(regularised_laplacian_kernel, G, csv_labeled_matrix_to_nparray(dir_path+'/04_unit_testing/regularisedLaplacianKernel.csv'))

INFO:root: Computed matrix  
 [[0.12776984 0.01982103 0.01521483 ... 0.0096302  0.00799348 0.00802052]
 [0.01982103 0.09644469 0.01419031 ... 0.029662   0.00731632 0.00934891]
 [0.01521483 0.01419031 0.05718463 ... 0.00781676 0.01878348 0.00724993]
 ...
 [0.0096302  0.029662   0.00781676 ... 0.28378943 0.00583329 0.00537881]
 [0.00799348 0.00731632 0.01878348 ... 0.00583329 0.27568824 0.00469274]
 [0.00802052 0.00934891 0.00724993 ... 0.00537881 0.00469274 0.26924707]]

INFO:root: Test matrix  
 [[0.12776984 0.01982103 0.01521483 ... 0.0096302  0.00799348 0.00802052]
 [0.01982103 0.09644469 0.01419031 ... 0.029662   0.00731632 0.00934891]
 [0.01521483 0.01419031 0.05718463 ... 0.00781676 0.01878348 0.00724993]
 ...
 [0.0096302  0.029662   0.00781676 ... 0.28378943 0.00583329 0.00537881]
 [0.00799348 0.00731632 0.01878348 ... 0.00583329 0.27568824 0.00469274]
 [0.00802052 0.00934891 0.00724993 ... 0.00537881 0.00469274 0.26924707]]

INFO:root: Test regularised_laplacian_kernel passed
